In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-feb-2021/train.csv
/kaggle/input/tabular-playground-series-feb-2021/test.csv


In [2]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot

In [3]:
data = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
data.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,A,B,A,A,B,D,A,E,C,...,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,B,A,A,A,B,B,A,E,A,...,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,A,A,A,C,B,D,A,B,C,...,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,A,A,A,C,B,D,A,E,G,...,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,A,B,A,A,B,B,A,E,C,...,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [4]:
for i in range(10):
    print(data['cat'+str(i)].value_counts())
    print()

A    281471
B     18529
Name: cat0, dtype: int64

A    162678
B    137322
Name: cat1, dtype: int64

A    276551
B     23449
Name: cat2, dtype: int64

C    183752
A    104464
D     11174
B       610
Name: cat3, dtype: int64

B    297373
A      1241
C       767
D       619
Name: cat4, dtype: int64

B    149208
D    135151
C     11763
A      3878
Name: cat5, dtype: int64

A    292643
B      6344
C       809
D       147
I        24
E        19
H        11
G         3
Name: cat6, dtype: int64

E    267631
D     24356
B      5750
G      1961
F       279
A        14
C         6
I         3
Name: cat7, dtype: int64

C    121054
E     94616
G     42195
A     37878
D      3694
F       549
B        14
Name: cat8, dtype: int64

F    107281
I     50064
L     42200
H     24759
K     20955
A     13408
G     10409
M      9838
J      6981
O      6173
N      4112
B      3435
C       209
D       103
E        73
Name: cat9, dtype: int64



In [5]:
# le = LabelEncoder()
# for col in data.columns:
#     if col.__contains__('cat'):
#         data[col] = le.fit_transform(data[col])
data = pd.get_dummies(data)
data.head()

,id,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,...,cat9_F,cat9_G,cat9_H,cat9_I,cat9_J,cat9_K,cat9_L,cat9_M,cat9_N,cat9_O
0,1,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,...,0,0,0,1,0,0,0,0,0,0
1,2,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,...,1,0,0,0,0,0,0,0,0,0
2,3,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,...,0,0,0,0,0,0,0,0,1,0
3,4,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,...,0,0,0,0,0,1,0,0,0,0
4,6,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,...,1,0,0,0,0,0,0,0,0,0


In [6]:
Ytrain = data['target']
Xtrain = data.drop(['target', 'id'], axis=1)

In [7]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()
# Don't cheat - fit only on training data
scaler.fit(Xtrain)  
X_train = scaler.transform(Xtrain)  


In [8]:
# # evaluate a given model using cross-validation
# def evaluate_model(model, X, y):
#     cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
#     scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
#     return scores

In [9]:
# CB_r = CatBoostRegressor()

In [10]:
# scores = evaluate_model(CB_r, Xtrain, Ytrain)

In [11]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


In [12]:
# define the base models
level0 = list()
level0.append(('cat', CatBoostRegressor()))
level0.append(('lgbm', LGBMRegressor()))
level0.append(('xgb', XGBRegressor()))
level0.append(('SGD', SGDRegressor(max_iter=1000, tol=1e-3)))


# define meta learner model
level1 = LinearRegression()

In [13]:
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
# fit the model on all available data
model.fit(X_train, Ytrain)


Learning rate set to 0.107728
0:	learn: 0.8839574	total: 96.7ms	remaining: 1m 36s
1:	learn: 0.8810760	total: 136ms	remaining: 1m 7s
2:	learn: 0.8786506	total: 180ms	remaining: 59.8s
3:	learn: 0.8765988	total: 237ms	remaining: 59s
4:	learn: 0.8749075	total: 273ms	remaining: 54.3s
5:	learn: 0.8733095	total: 312ms	remaining: 51.8s
6:	learn: 0.8718469	total: 352ms	remaining: 49.9s
7:	learn: 0.8705804	total: 385ms	remaining: 47.7s
8:	learn: 0.8695460	total: 421ms	remaining: 46.4s
9:	learn: 0.8685035	total: 459ms	remaining: 45.5s
10:	learn: 0.8675338	total: 501ms	remaining: 45s
11:	learn: 0.8666163	total: 538ms	remaining: 44.3s
12:	learn: 0.8657986	total: 578ms	remaining: 43.9s
13:	learn: 0.8650336	total: 617ms	remaining: 43.5s
14:	learn: 0.8643407	total: 655ms	remaining: 43s
15:	learn: 0.8636887	total: 691ms	remaining: 42.5s
16:	learn: 0.8630564	total: 729ms	remaining: 42.2s
17:	learn: 0.8624607	total: 773ms	remaining: 42.2s
18:	learn: 0.8619355	total: 813ms	remaining: 42s
19:	learn: 0.8614

165:	learn: 0.8437872	total: 6.71s	remaining: 33.7s
166:	learn: 0.8437283	total: 6.75s	remaining: 33.7s
167:	learn: 0.8436854	total: 6.79s	remaining: 33.6s
168:	learn: 0.8436297	total: 6.83s	remaining: 33.6s
169:	learn: 0.8435776	total: 6.86s	remaining: 33.5s
170:	learn: 0.8435328	total: 6.9s	remaining: 33.5s
171:	learn: 0.8434945	total: 6.95s	remaining: 33.4s
172:	learn: 0.8434524	total: 6.98s	remaining: 33.4s
173:	learn: 0.8433998	total: 7.02s	remaining: 33.3s
174:	learn: 0.8433596	total: 7.06s	remaining: 33.3s
175:	learn: 0.8433067	total: 7.09s	remaining: 33.2s
176:	learn: 0.8432627	total: 7.13s	remaining: 33.1s
177:	learn: 0.8432053	total: 7.17s	remaining: 33.1s
178:	learn: 0.8431389	total: 7.21s	remaining: 33.1s
179:	learn: 0.8430957	total: 7.24s	remaining: 33s
180:	learn: 0.8430508	total: 7.28s	remaining: 33s
181:	learn: 0.8430003	total: 7.32s	remaining: 32.9s
182:	learn: 0.8429580	total: 7.36s	remaining: 32.8s
183:	learn: 0.8429159	total: 7.4s	remaining: 32.8s
184:	learn: 0.8428

326:	learn: 0.8374771	total: 12.8s	remaining: 26.4s
327:	learn: 0.8374474	total: 12.9s	remaining: 26.4s
328:	learn: 0.8374084	total: 12.9s	remaining: 26.3s
329:	learn: 0.8373797	total: 12.9s	remaining: 26.3s
330:	learn: 0.8373570	total: 13s	remaining: 26.2s
331:	learn: 0.8373271	total: 13s	remaining: 26.2s
332:	learn: 0.8373000	total: 13.1s	remaining: 26.2s
333:	learn: 0.8372719	total: 13.1s	remaining: 26.1s
334:	learn: 0.8372378	total: 13.1s	remaining: 26.1s
335:	learn: 0.8372103	total: 13.2s	remaining: 26s
336:	learn: 0.8371775	total: 13.2s	remaining: 26s
337:	learn: 0.8371427	total: 13.2s	remaining: 25.9s
338:	learn: 0.8371118	total: 13.3s	remaining: 25.9s
339:	learn: 0.8370820	total: 13.3s	remaining: 25.9s
340:	learn: 0.8370523	total: 13.4s	remaining: 25.8s
341:	learn: 0.8370202	total: 13.4s	remaining: 25.8s
342:	learn: 0.8369816	total: 13.4s	remaining: 25.7s
343:	learn: 0.8369551	total: 13.5s	remaining: 25.7s
344:	learn: 0.8369243	total: 13.5s	remaining: 25.6s
345:	learn: 0.836886

487:	learn: 0.8326998	total: 19s	remaining: 19.9s
488:	learn: 0.8326786	total: 19s	remaining: 19.8s
489:	learn: 0.8326477	total: 19s	remaining: 19.8s
490:	learn: 0.8326227	total: 19.1s	remaining: 19.8s
491:	learn: 0.8325844	total: 19.1s	remaining: 19.7s
492:	learn: 0.8325594	total: 19.1s	remaining: 19.7s
493:	learn: 0.8325301	total: 19.2s	remaining: 19.6s
494:	learn: 0.8324971	total: 19.2s	remaining: 19.6s
495:	learn: 0.8324702	total: 19.2s	remaining: 19.6s
496:	learn: 0.8324425	total: 19.3s	remaining: 19.5s
497:	learn: 0.8324206	total: 19.3s	remaining: 19.5s
498:	learn: 0.8323902	total: 19.4s	remaining: 19.4s
499:	learn: 0.8323583	total: 19.4s	remaining: 19.4s
500:	learn: 0.8323262	total: 19.4s	remaining: 19.4s
501:	learn: 0.8322917	total: 19.5s	remaining: 19.3s
502:	learn: 0.8322613	total: 19.5s	remaining: 19.3s
503:	learn: 0.8322335	total: 19.5s	remaining: 19.2s
504:	learn: 0.8322026	total: 19.6s	remaining: 19.2s
505:	learn: 0.8321792	total: 19.6s	remaining: 19.1s
506:	learn: 0.8321

649:	learn: 0.8283990	total: 25s	remaining: 13.5s
650:	learn: 0.8283670	total: 25s	remaining: 13.4s
651:	learn: 0.8283477	total: 25.1s	remaining: 13.4s
652:	learn: 0.8283285	total: 25.1s	remaining: 13.3s
653:	learn: 0.8283064	total: 25.2s	remaining: 13.3s
654:	learn: 0.8282874	total: 25.2s	remaining: 13.3s
655:	learn: 0.8282670	total: 25.2s	remaining: 13.2s
656:	learn: 0.8282398	total: 25.3s	remaining: 13.2s
657:	learn: 0.8282217	total: 25.3s	remaining: 13.2s
658:	learn: 0.8282018	total: 25.3s	remaining: 13.1s
659:	learn: 0.8281704	total: 25.4s	remaining: 13.1s
660:	learn: 0.8281425	total: 25.4s	remaining: 13s
661:	learn: 0.8281178	total: 25.5s	remaining: 13s
662:	learn: 0.8280898	total: 25.5s	remaining: 13s
663:	learn: 0.8280647	total: 25.5s	remaining: 12.9s
664:	learn: 0.8280400	total: 25.6s	remaining: 12.9s
665:	learn: 0.8280235	total: 25.6s	remaining: 12.8s
666:	learn: 0.8280006	total: 25.7s	remaining: 12.8s
667:	learn: 0.8279659	total: 25.7s	remaining: 12.8s
668:	learn: 0.8279437	

...

499:	learn: 0.8311225	total: 16.5s	remaining: 16.5s
500:	learn: 0.8310852	total: 16.6s	remaining: 16.5s
501:	learn: 0.8310617	total: 16.6s	remaining: 16.5s
502:	learn: 0.8310279	total: 16.6s	remaining: 16.4s
503:	learn: 0.8309975	total: 16.7s	remaining: 16.4s
504:	learn: 0.8309673	total: 16.7s	remaining: 16.4s
505:	learn: 0.8309377	total: 16.7s	remaining: 16.3s
506:	learn: 0.8309017	total: 16.8s	remaining: 16.3s
507:	learn: 0.8308622	total: 16.8s	remaining: 16.3s
508:	learn: 0.8308321	total: 16.8s	remaining: 16.2s
509:	learn: 0.8308020	total: 16.9s	remaining: 16.2s
510:	learn: 0.8307667	total: 16.9s	remaining: 16.2s
511:	learn: 0.8307348	total: 16.9s	remaining: 16.1s
512:	learn: 0.8307016	total: 17s	remaining: 16.1s
513:	learn: 0.8306735	total: 17s	remaining: 16.1s
514:	learn: 0.8306430	total: 17s	remaining: 16s
515:	learn: 0.8306047	total: 17s	remaining: 16s
516:	learn: 0.8305660	total: 17.1s	remaining: 16s
517:	learn: 0.8305323	total: 17.1s	remaining: 15.9s
518:	learn: 0.8304957	tota

660:	learn: 0.8262243	total: 21.6s	remaining: 11.1s
661:	learn: 0.8262008	total: 21.6s	remaining: 11s
662:	learn: 0.8261673	total: 21.6s	remaining: 11s
663:	learn: 0.8261392	total: 21.7s	remaining: 11s
664:	learn: 0.8261175	total: 21.7s	remaining: 10.9s
665:	learn: 0.8260915	total: 21.7s	remaining: 10.9s
666:	learn: 0.8260663	total: 21.8s	remaining: 10.9s
667:	learn: 0.8260446	total: 21.8s	remaining: 10.8s
668:	learn: 0.8260138	total: 21.8s	remaining: 10.8s
669:	learn: 0.8259845	total: 21.9s	remaining: 10.8s
670:	learn: 0.8259534	total: 21.9s	remaining: 10.7s
671:	learn: 0.8259274	total: 21.9s	remaining: 10.7s
672:	learn: 0.8259023	total: 21.9s	remaining: 10.7s
673:	learn: 0.8258765	total: 22s	remaining: 10.6s
674:	learn: 0.8258426	total: 22s	remaining: 10.6s
675:	learn: 0.8258072	total: 22s	remaining: 10.6s
676:	learn: 0.8257734	total: 22.1s	remaining: 10.5s
677:	learn: 0.8257455	total: 22.1s	remaining: 10.5s
678:	learn: 0.8257251	total: 22.1s	remaining: 10.5s
679:	learn: 0.8256928	to

824:	learn: 0.8217261	total: 26.7s	remaining: 5.67s
825:	learn: 0.8216973	total: 26.8s	remaining: 5.64s
826:	learn: 0.8216724	total: 26.8s	remaining: 5.6s
827:	learn: 0.8216454	total: 26.8s	remaining: 5.57s
828:	learn: 0.8216260	total: 26.9s	remaining: 5.54s
829:	learn: 0.8215892	total: 26.9s	remaining: 5.51s
830:	learn: 0.8215551	total: 26.9s	remaining: 5.47s
831:	learn: 0.8215391	total: 27s	remaining: 5.44s
832:	learn: 0.8215087	total: 27s	remaining: 5.41s
833:	learn: 0.8214772	total: 27s	remaining: 5.38s
834:	learn: 0.8214531	total: 27s	remaining: 5.34s
835:	learn: 0.8214241	total: 27.1s	remaining: 5.31s
836:	learn: 0.8214047	total: 27.1s	remaining: 5.28s
837:	learn: 0.8213763	total: 27.1s	remaining: 5.25s
838:	learn: 0.8213541	total: 27.2s	remaining: 5.21s
839:	learn: 0.8213300	total: 27.2s	remaining: 5.18s
840:	learn: 0.8212947	total: 27.2s	remaining: 5.15s
841:	learn: 0.8212699	total: 27.3s	remaining: 5.12s
842:	learn: 0.8212416	total: 27.3s	remaining: 5.08s
843:	learn: 0.8212142

984:	learn: 0.8173303	total: 31.7s	remaining: 483ms
985:	learn: 0.8173037	total: 31.8s	remaining: 451ms
986:	learn: 0.8172690	total: 31.8s	remaining: 419ms
987:	learn: 0.8172341	total: 31.9s	remaining: 387ms
988:	learn: 0.8171991	total: 31.9s	remaining: 355ms
989:	learn: 0.8171698	total: 31.9s	remaining: 322ms
990:	learn: 0.8171413	total: 32s	remaining: 290ms
991:	learn: 0.8171150	total: 32s	remaining: 258ms
992:	learn: 0.8170842	total: 32s	remaining: 226ms
993:	learn: 0.8170626	total: 32s	remaining: 193ms
994:	learn: 0.8170426	total: 32.1s	remaining: 161ms
995:	learn: 0.8170131	total: 32.1s	remaining: 129ms
996:	learn: 0.8169848	total: 32.1s	remaining: 96.7ms
997:	learn: 0.8169533	total: 32.2s	remaining: 64.5ms
998:	learn: 0.8169302	total: 32.2s	remaining: 32.2ms
999:	learn: 0.8168979	total: 32.2s	remaining: 0us


StackingRegressor(cv=5,
                  estimators=[('cat',
                               <catboost.core.CatBoostRegressor object at 0x7fd631c7ef90>),
                              ('lgbm', LGBMRegressor()),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
           

In [15]:
model.score(X_train, Ytrain)

0.142949589143211

In [16]:
test_data = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
test_data.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,A,B,A,C,B,D,A,E,E,...,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448
1,5,A,B,A,C,B,D,A,E,C,...,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251
2,15,A,B,A,C,B,D,A,E,C,...,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905
3,16,A,A,B,A,B,D,A,E,E,...,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625
4,17,A,B,A,A,B,B,A,E,E,...,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931


In [17]:
# for col in test_data.columns:
#     if col.__contains__('cat'):
#         test_data[col] = le.fit_transform(test_data[col])
test_data = pd.get_dummies(test_data)
test_data.head()

,id,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,...,cat9_F,cat9_G,cat9_H,cat9_I,cat9_J,cat9_K,cat9_L,cat9_M,cat9_N,cat9_O
0,0,0.331261,0.551131,0.486284,0.156058,0.701679,0.595507,0.286912,0.279884,0.202234,...,0,1,0,0,0,0,0,0,0,0
1,5,0.402404,0.617546,0.296852,0.187440,0.277480,0.479552,0.397436,0.476742,0.857073,...,0,0,0,0,0,0,1,0,0,0
2,15,0.530677,0.618126,0.760865,0.199872,0.279508,0.676395,0.695284,0.253316,0.586934,...,1,0,0,0,0,0,0,0,0,0
3,16,0.439653,0.487403,0.715786,0.192808,0.479503,0.759875,0.240049,0.298074,0.442475,...,1,0,0,0,0,0,0,0,0,0
4,17,0.389844,0.752760,0.315269,0.191734,0.757845,0.210232,0.329851,0.616663,0.170475,...,0,0,0,1,0,0,0,0,0,0


In [18]:
(set(data.columns)) - (set(test_data.columns))

{'cat6_G', 'target'}

In [19]:
test_data['cat6_G'] = 0

In [20]:
Xtrain.columns

Index(['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cat0_A',
       'cat0_B', 'cat1_A', 'cat1_B', 'cat2_A', 'cat2_B', 'cat3_A', 'cat3_B',
       'cat3_C', 'cat3_D', 'cat4_A', 'cat4_B', 'cat4_C', 'cat4_D', 'cat5_A',
       'cat5_B', 'cat5_C', 'cat5_D', 'cat6_A', 'cat6_B', 'cat6_C', 'cat6_D',
       'cat6_E', 'cat6_G', 'cat6_H', 'cat6_I', 'cat7_A', 'cat7_B', 'cat7_C',
       'cat7_D', 'cat7_E', 'cat7_F', 'cat7_G', 'cat7_I', 'cat8_A', 'cat8_B',
       'cat8_C', 'cat8_D', 'cat8_E', 'cat8_F', 'cat8_G', 'cat9_A', 'cat9_B',
       'cat9_C', 'cat9_D', 'cat9_E', 'cat9_F', 'cat9_G', 'cat9_H', 'cat9_I',
       'cat9_J', 'cat9_K', 'cat9_L', 'cat9_M', 'cat9_N', 'cat9_O'],
      dtype='object')

In [21]:
test_data.columns[1:]

Index(['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cat0_A',
       'cat0_B', 'cat1_A', 'cat1_B', 'cat2_A', 'cat2_B', 'cat3_A', 'cat3_B',
       'cat3_C', 'cat3_D', 'cat4_A', 'cat4_B', 'cat4_C', 'cat4_D', 'cat5_A',
       'cat5_B', 'cat5_C', 'cat5_D', 'cat6_A', 'cat6_B', 'cat6_C', 'cat6_D',
       'cat6_E', 'cat6_H', 'cat6_I', 'cat7_A', 'cat7_B', 'cat7_C', 'cat7_D',
       'cat7_E', 'cat7_F', 'cat7_G', 'cat7_I', 'cat8_A', 'cat8_B', 'cat8_C',
       'cat8_D', 'cat8_E', 'cat8_F', 'cat8_G', 'cat9_A', 'cat9_B', 'cat9_C',
       'cat9_D', 'cat9_E', 'cat9_F', 'cat9_G', 'cat9_H', 'cat9_I', 'cat9_J',
       'cat9_K', 'cat9_L', 'cat9_M', 'cat9_N', 'cat9_O', 'cat6_G'],
      dtype='object')

In [22]:
for i, col in enumerate(test_data.columns):
    print(i, ' ', col)

0   id
1   cont0
2   cont1
3   cont2
4   cont3
5   cont4
6   cont5
7   cont6
8   cont7
9   cont8
10   cont9
11   cont10
12   cont11
13   cont12
14   cont13
15   cat0_A
16   cat0_B
17   cat1_A
18   cat1_B
19   cat2_A
20   cat2_B
21   cat3_A
22   cat3_B
23   cat3_C
24   cat3_D
25   cat4_A
26   cat4_B
27   cat4_C
28   cat4_D
29   cat5_A
30   cat5_B
31   cat5_C
32   cat5_D
33   cat6_A
34   cat6_B
35   cat6_C
36   cat6_D
37   cat6_E
38   cat6_H
39   cat6_I
40   cat7_A
41   cat7_B
42   cat7_C
43   cat7_D
44   cat7_E
45   cat7_F
46   cat7_G
47   cat7_I
48   cat8_A
49   cat8_B
50   cat8_C
51   cat8_D
52   cat8_E
53   cat8_F
54   cat8_G
55   cat9_A
56   cat9_B
57   cat9_C
58   cat9_D
59   cat9_E
60   cat9_F
61   cat9_G
62   cat9_H
63   cat9_I
64   cat9_J
65   cat9_K
66   cat9_L
67   cat9_M
68   cat9_N
69   cat9_O
70   cat6_G


In [23]:
col_names = ['id']
for col in Xtrain.columns.values:
    col_names.append(col)
print(col_names)

['id', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cat0_A', 'cat0_B', 'cat1_A', 'cat1_B', 'cat2_A', 'cat2_B', 'cat3_A', 'cat3_B', 'cat3_C', 'cat3_D', 'cat4_A', 'cat4_B', 'cat4_C', 'cat4_D', 'cat5_A', 'cat5_B', 'cat5_C', 'cat5_D', 'cat6_A', 'cat6_B', 'cat6_C', 'cat6_D', 'cat6_E', 'cat6_G', 'cat6_H', 'cat6_I', 'cat7_A', 'cat7_B', 'cat7_C', 'cat7_D', 'cat7_E', 'cat7_F', 'cat7_G', 'cat7_I', 'cat8_A', 'cat8_B', 'cat8_C', 'cat8_D', 'cat8_E', 'cat8_F', 'cat8_G', 'cat9_A', 'cat9_B', 'cat9_C', 'cat9_D', 'cat9_E', 'cat9_F', 'cat9_G', 'cat9_H', 'cat9_I', 'cat9_J', 'cat9_K', 'cat9_L', 'cat9_M', 'cat9_N', 'cat9_O']


In [24]:
test_data = test_data.reindex(columns=col_names)

In [25]:
test_df = test_data.drop(['id'], axis=1)
# apply same transformation to test data
X_test = scaler.transform(test_df)  
prediction = model.predict(X_test)
test_data['target'] = prediction
test_data.head()

,id,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,...,cat9_G,cat9_H,cat9_I,cat9_J,cat9_K,cat9_L,cat9_M,cat9_N,cat9_O,target
0,0,0.331261,0.551131,0.486284,0.156058,0.701679,0.595507,0.286912,0.279884,0.202234,...,1,0,0,0,0,0,0,0,0,7.638267
1,5,0.402404,0.617546,0.296852,0.187440,0.277480,0.479552,0.397436,0.476742,0.857073,...,0,0,0,0,0,1,0,0,0,7.772018
2,15,0.530677,0.618126,0.760865,0.199872,0.279508,0.676395,0.695284,0.253316,0.586934,...,0,0,0,0,0,0,0,0,0,7.621819
3,16,0.439653,0.487403,0.715786,0.192808,0.479503,0.759875,0.240049,0.298074,0.442475,...,0,0,0,0,0,0,0,0,0,7.482527
4,17,0.389844,0.752760,0.315269,0.191734,0.757845,0.210232,0.329851,0.616663,0.170475,...,0,0,1,0,0,0,0,0,0,7.303098


In [26]:
for col in test_data.columns:
    if col.__contains__('cat') or col.__contains__('cont'):
        test_data.drop([col], axis=1, inplace=True)
test_data.head()

,id,target
0,0,7.638267
1,5,7.772018
2,15,7.621819
3,16,7.482527
4,17,7.303098


In [27]:
test_data.to_csv('submission.csv', index=False)